In [1]:
# %%capture
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
# !conda install pyg -c pyg
# !conda install gemmi -c conda-forge
# !conda install biopython
# !pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
# !pip install torch-geometric
# !pip install wandb
# !pip install plotly


Torch version: 2.4.0+cu124
CUDA available:  True
Torch geometric version: 2.6.1


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: mwfjord to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


structural_binding_affinity_predictions_using_gnn


In [4]:
batch_size = config.training.batch_size

# Load preprocessed datasets
train_dataset = ProteinDNADataset("../data")
val_dataset = ProteinDNADataset("../data", validation=True)
test_dataset = ProteinDNADataset("../data", test=True)

# print("Training sample:", train_dataset[0])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


data_details = {
    "num_node_features": train_dataset.num_node_features,
    "num_edge_features": train_dataset.num_edge_features,
    "num_classes": train_dataset.num_classes,
    "num_training_samples": len(train_dataset),
    "num_validation_samples": len(val_dataset),
    "num_test_samples": len(test_dataset),
}


# Log the dataset to W&B as an artifact.
dataset_artifact = wandb.Artifact(name="KLF1_K_d", type="dataset", metadata=data_details)
dataset_artifact.add_dir("../data")
wandb.log_artifact(dataset_artifact)



/home/marcus/programming/structural_binding_affinity_predictions_using_gnn/src/proteinDNADataset.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(os.p

<Artifact KLF1_K_d>

# Initialize the model

4
Using GPU


/home/marcus/programming/structural_binding_affinity_predictions_using_gnn/src/proteinDNADataset.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(os.p

In [9]:
epochs = 10  # You can increase this based on your dataset size

for epoch in range(1, epochs + 1):
    
    train_loss = train()
    val_loss = validate()
    
    wandb.log({
            "train/loss": train_loss,
            "val/loss": val_loss,
            "epoch": epoch
        })
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "./trained_models/protein_dna_gnn_model.pth")
artifact = wandb.Artifact(name="graph_classification_model", type="model")
artifact.add_file("./trained_models/protein_dna_gnn_model.pth")
wandb.log_artifact(artifact)
wandb.finish()

Epoch 1: Train Loss = 432523544.8889, Val Loss = 450496608.0000
Epoch 10: Train Loss = 429904672.0000, Val Loss = 450488096.0000


epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇██▁
train/loss,██▇▇▆▃▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
val/loss,█▇▇▅▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,10
train/loss,429904672.0
val/loss,450488096.0


In [ ]:
def test():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y)
            total_loss += loss.item()
    return total_loss / len(test_loader)

test_loss = test()
print(f"Test Loss: {test_loss:.4f}")


# Make Predictions on New Data

In [ ]:
def predict(data):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        out = model(data.x, data.edge_index, data.batch)
    return out.item()

# Example prediction on a single protein-DNA complex
sample_data = test_dataset[0]  # Take the first test sample
predicted_affinity = predict(sample_data)
print(f"Predicted Binding Affinity: {predicted_affinity:.4f}")
